In [4]:
import pandas as pd
import numpy as np
import zipfile
import plotly.express as px
import json
import requests, io
import geopandas as gpd
from us import states


In [2]:
zip_path = 'clean_data\soc_chas_redfin.zip'
csv_inside = 'soc_chas_redfin.csv'

In [5]:
with zipfile.ZipFile(zip_path) as z:
    with z.open(csv_inside) as f:
        merged = pd.read_csv(f)
        
merged['COUNTY_FIPS'] = merged['GEOID'].astype(str).str.zfill(5)

In [ ]:
numeric_cols = [
    "MEDIAN_SALE_PRICE",
    "median_household_income",
    "poverty_rate_pct",
    "unemployment_rate_pct",
    "Percent_of_low_income_cb_over30_percent",
    "Percent_of_low_income_cb_over50_percent",
    "Percent_of_medi_income_cb_over30_percent",
    "Percent_of_medi_income_cb_over50_percent",
]
merged[numeric_cols] = merged[numeric_cols].apply(pd.to_numeric, errors="coerce")

geojson_url = (
    "https://raw.githubusercontent.com/plotly/datasets/master/"
    "geojson-counties-fips.json"
)
counties_geojson = requests.get(geojson_url).json()

afford_cols = {
    "MEDIAN_SALE_PRICE"                       : "$,.0f",
    "median_household_income"                 : "$,.0f",
    "poverty_rate_pct"                        : ".2f",
    "unemployment_rate_pct"                   : ".2f",
    "Percent_of_low_income_cb_over30_percent" : ".2f",
    "Percent_of_low_income_cb_over50_percent" : ".2f",
    "Percent_of_medi_income_cb_over30_percent": ".2f",
    "Percent_of_medi_income_cb_over50_percent": ".2f",
}
hover_dict = {c: f":{fmt}" for c, fmt in afford_cols.items()}
hover_dict.update({"NAME": True, "STATE_CODE": True})  

fig = px.choropleth(
    merged,
    geojson=counties_geojson,
    locations="COUNTY_FIPS",
    color="MEDIAN_SALE_PRICE",
    color_continuous_scale="Reds",
    range_color=(
        merged["MEDIAN_SALE_PRICE"].min(),
        merged["MEDIAN_SALE_PRICE"].quantile(0.95),
    ),
    scope="usa",
    hover_data=hover_dict,
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title_text="Average 2016–20 Redfin Median Sale Price by County",
    margin=dict(r=0, t=40, l=0, b=0),
    coloraxis_colorbar_title="",
)

fig.show()